In [2]:
import torch
import numpy as np
import pandas as pd

# Função kernel (RBF ajustado com a equação 21.39)
def Ridgkernel(x1, x2, sigma_0=1.0, l=1.0):
    diff = np.array(x1, dtype=float) - np.array(x2, dtype=float)
    return sigma_0**2 * np.exp(-np.dot(diff, diff) / (2 * l**2))

# Implementação da Regressão Ridge com SGD para Séries Temporais Online usando PyTorch
class RidgeSGDKernelTorchDict:
    def __init__(self, eta=0.01, c=1.0, sigma=1.0):
        self.eta = eta  # taxa de aprendizado
        self.c = c  # parâmetro de regularização
        self.sigma = sigma  # parâmetro do kernel
        self.alpha_dict = {}  # dicionário para armazenar os pesos associados às chaves
        self.X_train_dict = {}  # dicionário para armazenar os dados de treino

    def partial_fit(self, x_new, y_new, key, iteration):
        """
        Adiciona uma nova entrada ao modelo e atualiza os pesos.
        
        Args:
        - x_new: nova amostra (array ou lista de características).
        - y_new: valor alvo associado à amostra.
        - key: identificador único para a amostra.
        - iteration: número da iteração atual.
        """
        print(f"\nInteração {iteration} - Método: partial_fit")
        x_new_tensor = torch.tensor(x_new, dtype=torch.float32)
        self.X_train_dict[key] = x_new_tensor  # Adiciona nova entrada ao conjunto de treino
        if key not in self.alpha_dict:
            self.alpha_dict[key] = torch.tensor(0.0, dtype=torch.float32)  # Inicializa o peso correspondente

        # Calcular a previsão antes de ajustar o modelo
        pred_n = self.predict(x_new, iteration)
        print(f"  Antes da atualização: Previsão para {key} = {pred_n}")

        # Calcular o erro
        error = y_new - pred_n

        # Atualizar os pesos
        for k in list(self.alpha_dict.keys()):
            if k != key:
                self.alpha_dict[k] *= (1 - self.eta * self.c)  # Atualização regularizada
        self.alpha_dict[key] = self.eta * error  # Atualiza o peso para a nova amostra

        print(f"  Pesos atualizados após a amostra {key}: {self.alpha_dict}")

    def predict(self, x_new, iteration):
        """
        Realiza uma previsão para uma nova entrada.
        
        Args:
        - x_new: nova entrada (array ou lista de características).
        - iteration: número da iteração atual.
        
        Returns:
        - Previsão como float.
        """
        print(f"\nInteração {iteration} - Método: predict")
        x_new_tensor = torch.tensor(x_new, dtype=torch.float32)
        prediction = 0
        for k, x_train in self.X_train_dict.items():
            kernel_value = torch.exp(-torch.norm(x_train - x_new_tensor) ** 2 / (2 * self.sigma ** 2))
            prediction += self.alpha_dict[k] * kernel_value
        print(f"  Previsão feita para {x_new}: {prediction.item()}")
        return prediction.item()

# Função para aplicar o modelo e obter previsões e resíduos
def apply_model_and_plot(X_residual, y_residual):
    # Converter os dados em dicionários para serem compatíveis com o modelo
    X_residual_dict = {idx: x for idx, x in zip(y_residual.index, X_residual.values)}
    y_residual_dict = {idx: y for idx, y in zip(y_residual.index, y_residual.values)}

    # Instanciar o modelo com os parâmetros desejados
    ridge_sgd_torch = RidgeSGDKernelTorchDict(eta=0.68, c=1, sigma=0.9)

    # Lista para armazenar previsões
    y_pred = {}

    # Treinar e prever de forma online com os dados residuais
    for iteration, idx in enumerate(X_residual_dict.keys(), 1):
        x_new = X_residual_dict[idx]
        y_new = y_residual_dict[idx]
        # Depois, atualizar o modelo com a nova amostra
        ridge_sgd_torch.partial_fit(x_new, y_new, idx, iteration)
        y_pred[idx] = ridge_sgd_torch.predict(x_new, iteration)

    # Converter as previsões em um pandas.Series para facilitar o plot
    y_pred_series = pd.Series(y_pred).sort_index()
    y_residual_series = pd.Series(y_residual_dict).sort_index()

    return y_residual_series, y_pred_series

# Exemplo de uso com os dados fornecidos
X_residual = [1.5, 1.5, 2.5, 3.5]
y_residual = [1.5, 2.5, 3.5, 4.5]
y_residual_series, y_pred_series = apply_model_and_plot(pd.Series(X_residual), pd.Series(y_residual))

print(f"Previsões: \n{y_pred_series}")
print(f"Resíduos: \n{y_residual_series}")




Interação 1 - Método: partial_fit

Interação 1 - Método: predict
  Previsão feita para 1.5: 0.0
  Antes da atualização: Previsão para 0 = 0.0
  Pesos atualizados após a amostra 0: {0: 1.02}

Interação 1 - Método: predict
  Previsão feita para 1.5: 1.0199999809265137

Interação 2 - Método: partial_fit

Interação 2 - Método: predict
  Previsão feita para 1.5: 1.0199999809265137
  Antes da atualização: Previsão para 1 = 1.0199999809265137
  Pesos atualizados após a amostra 1: {0: 0.32639999999999997, 1: 1.0064000129699708}

Interação 2 - Método: predict
  Previsão feita para 1.5: 1.332800030708313

Interação 3 - Método: partial_fit

Interação 3 - Método: predict
  Previsão feita para 2.5: 0.7189222574234009
  Antes da atualização: Previsão para 2 = 0.7189222574234009
  Pesos atualizados após a amostra 2: {0: 0.10444799999999997, 1: 0.32204800415039064, 2: 1.8911328649520875}

Interação 3 - Método: predict
  Previsão feita para 2.5: 2.121187925338745

Interação 4 - Método: partial_fit

In